In [1]:
import premise
import bw2data as bd
from bw2data import Database, Method
import bw2io as bi 
import brightway2 as bw
import numpy as np

from config import * #some main key parameters needed.
from private_keys import * #import secret keys
from db_functions import * # this is a python file with self-created functions needed to run this notebook
from functions import * # this is a python file with self-created functions needed to run this notebook
premise.__version__

(2, 2, 7)

In [2]:
# Delete project by name
#projects.delete_project('trails_v0.1', delete_dir=True)
#projects.purge_deleted_directories()

In [3]:
#list(bw.projects)

In [4]:
#bw.projects.delete_project('lca_steel_cbam', delete_dir=True)
#projects.purge_deleted_directories()

In [5]:
GENERATE_ACTS=True

In [6]:
# Import project
bw.projects.set_current(PROJECT_NAME) #Creating/accessing the project
DB_NAME_INIT,PROJECT_NAME

('ecoinvent-3.10-cutoff', 'steel_sector_cbam')

In [7]:
# Create dbs if not existing yet
if DB_NAME_INIT not in bw.databases:
    bi.import_ecoinvent_release(EI_VERSION, "cutoff", USER_NAME, USER_PW)

In [8]:
flows_cbam = []
flow_name = []

cbam_method_name = ('CBAM', 'co2 emissions', 'co2 emissions')

#if cbam_method_name not in bd.methods:
for flow_key, cf in bw.Method(
    ('EF v3.1 EN15804', 'climate change', 'global warming potential (GWP100)')
).load():

    bio = bw.get_activity(flow_key)  # <-- THIS is the missing step

    if (
        "Carbon dioxide" in bio['name']
        and 'air' in str(bio['categories'])
        and bio['unit'] == "kilogram" and cf==1
    ):
        flows_cbam.append((flow_key, cf))
        print(bio['name'], bio['categories'], bio['unit'])
        flow_name.append((bio['name'], bio['categories'], bio['unit']))

my_method = bw.Method(cbam_method_name)
my_method.register(
    unit="kg CO2",
    meaning="carbon dioxide emissions only, for CBAM assessment"
)

my_method.write(flows_cbam)

Carbon dioxide, fossil ('air', 'low population density, long-term') kilogram
Carbon dioxide, fossil ('air', 'lower stratosphere + upper troposphere') kilogram
Carbon dioxide, fossil ('air', 'non-urban air or from high stacks') kilogram
Carbon dioxide, fossil ('air',) kilogram
Carbon dioxide, fossil ('air', 'urban air close to ground') kilogram
Carbon dioxide, from soil or biomass stock ('air', 'low population density, long-term') kilogram
Carbon dioxide, from soil or biomass stock ('air', 'non-urban air or from high stacks') kilogram
Carbon dioxide, from soil or biomass stock ('air',) kilogram
Carbon dioxide, from soil or biomass stock ('air', 'urban air close to ground') kilogram
Carbon dioxide, non-fossil ('air', 'low population density, long-term') kilogram
Carbon dioxide, non-fossil ('air', 'non-urban air or from high stacks') kilogram
Carbon dioxide, non-fossil ('air',) kilogram
Carbon dioxide, non-fossil ('air', 'urban air close to ground') kilogram


In [9]:
flows = []

if MY_METHODS[0] not in bd.methods:
    # Start from the IPCC 2021 method
    for flow in bw.Method(('EF v3.1 EN15804', 'climate change', 'global warming potential (GWP100)')).load():
        flows.append(flow)
    
    # finally, add hydrogen flows
    for bio in bw.Database(BIOSPHERE_DB):
        if "Hydrogen" == bio['name'] and 'air' in str(bio['categories']) and bio['unit'] == "kilogram":
            bio_key = bio[1]
            print(bio)
            flows.append(((BIOSPHERE_DB, bio_key), 11)) # use a GWP of 11 for H2, according to premise-gwp and other refs.
    
    # write flows
    my_method = Method(MY_METHODS[0])
    my_metadata = {"unit": "kg CO2-eq.", "meaning": "to represent climate change"}
    my_method.register(**my_metadata)
        
    bw.Method(MY_METHODS[0]).write(flows)

In [10]:
list_spec_scenarios, list_names = generate_future_ei_dbs(scenarios = ["SSP2-PkBudg1150"], iam = 'remind',
                                                         start_yr=2025, end_yr = 2050, step = 5, endstring="base")
if len(list_spec_scenarios)>0:
    generate_prospective_lca_dbs(list_spec_scenarios, list_names)

Avoid duplicated db and therefore following db not added: 'ecoinvent_remind_SSP2-PkBudg1150_2030_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_remind_SSP2-PkBudg1150_2035_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_remind_SSP2-PkBudg1150_2040_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_remind_SSP2-PkBudg1150_2045_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_remind_SSP2-PkBudg1150_2050_base'


In [11]:
list_spec_scenarios, list_names = generate_future_ei_dbs(scenarios = ["SSP2-RCP26"], iam = 'image',
                                                         start_yr=2025, end_yr = 2050, step = 5, endstring="base")
if len(list_spec_scenarios)>0:
    generate_prospective_lca_dbs(list_spec_scenarios, list_names)

Avoid duplicated db and therefore following db not added: 'ecoinvent_image_SSP2-RCP26_2030_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_image_SSP2-RCP26_2035_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_image_SSP2-RCP26_2040_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_image_SSP2-RCP26_2045_base'
Avoid duplicated db and therefore following db not added: 'ecoinvent_image_SSP2-RCP26_2050_base'


In [12]:
all_ei_databases = [db_2 for db_2 in bw.databases if 'ecoinvent_' in str(db_2) or NAME_REF_DB in str(db_2)]
all_ei_databases

['ecoinvent_310_reference',
 'ecoinvent_remind_SSP2-PkBudg1150_2030_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2035_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2040_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2045_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2050_base',
 'ecoinvent_image_SSP2-RCP26_2030_base',
 'ecoinvent_image_SSP2-RCP26_2035_base',
 'ecoinvent_image_SSP2-RCP26_2040_base',
 'ecoinvent_image_SSP2-RCP26_2045_base',
 'ecoinvent_image_SSP2-RCP26_2050_base']

In [13]:
cf_hydro = 0.8 # IEA, excel sheet of hydrogen production facilities
cf_grid = 0.57 # IEA, excel sheet of hydrogen production facilities
cf_nuclear = 0.8 # IEA, excel sheet of hydrogen production facilities

if GENERATE_ACTS: 
    #loop over generated dbs of premise
    for db in all_ei_databases:
        print(db)
        for electr in ["pem"]: #, "aec", "soec"]: we only use pem now for simplification
            print(electr)
            list_cfs_solar = [round(x,3) for x in np.arange(0.10, 0.30, 0.025)]
            list_cfs_wind = [round(x,2) for x in np.arange(0.2, 0.60, 0.05)]
            
            for x in list_cfs_solar:
                curtail_pv = curve_fit_f(x, db, "ratio_pv_curtailed")
                # generate h2 production activities for ground-mounted solar PV
                GENERATE_ACTS_GM_PV(db, x, curtail_pv, cf_electrolyzer=0.3, electrolyzer = electr,
                                   excel_col_name = db)
                
            for x in list_cfs_wind:
                #generate activities for onshore and offshore wind
                curtail_wind_on = curve_fit_f(x, db, "ratio_wind_curtailed_on")
                GENERATE_ACTS_WIND(db, x, curtail_wind_on, cf_electrolyzer=0.4, electrolyzer = electr,
                                   excel_col_name = db)
            
            curtail_wind_off = 0
            GENERATE_ACTS_WIND_OFF(db, 0.55, curtail_wind_off, cf_electrolyzer=0.55, electrolyzer = electr,
                               excel_col_name = db)
        
            # Make hydropower based H2 production activities for water electrolysis
            # Here, we assume pumper hydro is this is the largest growing hydropower source, see:
            # https://iea.blob.core.windows.net/assets/4d2d4365-08c6-4171-9ea2-8549fabd1c8d/HydropowerSpecialMarketReport_corr.pdf
            print("Making hydro acts")
            act_hydro = ["electricity production, hydro, reservoir, alpine region", 'electricity, high voltage']
            
            all_db_locs_hy = [act['location'] for act in bw.Database(db) 
                               if act_hydro[0] == act['name'] and act_hydro[1] == act['reference product'] 
                           and len(act['location'])<=3]   # to only get countries and not provinces 
    
            # Loop over locations and generate them
            for loc in all_db_locs_hy:
                GENERATE_ACTS_GENERIC_ELECT(db, cf_electrolyzer=cf_hydro, electrolyzer = electr,
                                                generic_electr_act = [act_hydro[0],
                                                                      act_hydro[1],
                                                                      loc], excel_col_name = db)
                                                
            # Make power grid based H2 production activities for water electrolysis    
            print("Making power acts")
            act_grid = ['market for electricity, medium voltage', 'electricity, medium voltage']
            
            all_db_locs_grid = [act['location'] for act in bw.Database(db) 
                               if act_grid[0] == act['name'] and act_grid[1] == act['reference product'] 
                           and len(act['location'])<=3]  # to only get countries and not provinces 
            
            for loc in all_db_locs_grid:
                GENERATE_ACTS_GENERIC_ELECT(db, cf_electrolyzer=cf_grid, electrolyzer = electr,
                                                generic_electr_act = [act_grid[0],
                                                                      act_grid[1],
                                                                      loc], excel_col_name = db)
    
            # For aggregated market group power zones for grid-based water electrolysis
            act_grid = ['market group for electricity, medium voltage', 'electricity, medium voltage'] 
            
            all_db_locs_grid = [act['location'] for act in bw.Database(db) 
                               if act_grid[0] == act['name'] and act_grid[1] == act['reference product'] 
                           and len(act['location'])<=3]  # to only get countries and not provinces 
            
            for loc in all_db_locs_grid:
                GENERATE_ACTS_GENERIC_ELECT(db, cf_electrolyzer=cf_grid, electrolyzer = electr,
                                                generic_electr_act = [act_grid[0],
                                                                      act_grid[1],
                                                                      loc], excel_col_name = db)
    
            # For water electrolysis using power from nuclear power plants
            # larges from PWR: https://world-nuclear.org/nuclear-essentials/are-there-different-types-of-reactor, assume that here.
            print("Making nuclear acts")
            act_nuclear = ['electricity production, nuclear, pressure water reactor', 'electricity, high voltage'] 
            
            all_db_locs_nucl = [act['location'] for act in bw.Database(db) 
                               if act_nuclear[0] == act['name'] and act_nuclear[1] == act['reference product'] 
                           and len(act['location'])<=3]  # to only get countries and not provinces 
            
            for loc in all_db_locs_nucl:#[:3]:
                GENERATE_ACTS_GENERIC_ELECT(db, cf_electrolyzer=cf_nuclear, electrolyzer = electr,
                                                generic_electr_act = [act_nuclear[0],
                                                                      act_nuclear[1],
                                                                      loc], excel_col_name = db)

ecoinvent_310_reference
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2030_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2035_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2040_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2045_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2050_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_image_SSP2-RCP26_2030_base
pem
Making hydro acts
Making power acts
Skipped: activity 'hydrogen production, gaseous, 30 bar, from PEM electrolysis, power from market for electricity, medium voltage' already generated!
Making nuclear acts
ecoinvent_image_SSP2-RCP26_2035_base
pem
Making hydro acts
Making power acts
Skipped: activity 'hydrogen production, gaseous, 30 bar, from PEM electrolysi

ecoinvent_remind_SSP2-PkBudg1150_2030_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2035_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2040_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2045_base
pem
Making hydro acts
Making power acts
Making nuclear acts
ecoinvent_remind_SSP2-PkBudg1150_2050_base
pem
Making hydro acts
Making power acts

In [14]:
# Create a new LCIA method for water consumption based on recipe 2016 
# https://github.com/brightway-lca/bw_recipe_2016/blob/master/bw_recipe_2016/categories/water.py
new_water_name = ("Own method", "Water consumption", "Water consumption")

if new_water_name not in bw.methods:
    my_cfs_water = []
    
    FLOWS = [
        ("Water", ("water",)),
        ("Water", ("water", "fossil well")),
        ("Water", ("water", "ground-")),
        ("Water", ("water", "ground-, long-term")),
        ("Water", ("water", "surface water")),
        ("Water, cooling, unspecified natural origin", ("natural resource", "in water")),
        ("Water, lake", ("natural resource", "in water")),
        ("Water, river", ("natural resource", "in water")),
        (
            "Water, turbine use, unspecified natural origin",
            ("natural resource", "in water"),
        ),
        # ('Water, unspecified natural origin', ('natural resource', 'fossil well')),
        ("Water, unspecified natural origin", ("natural resource", "in ground")),
        ("Water, unspecified natural origin", ("natural resource", "in water")),
        ("Water, well, in ground", ("natural resource", "in water")),
    ]
    
    for bio in bw.Database(BIOSPHERE_DB):
        bio_check_name = (bio['name'], bio['categories'])
        if bio_check_name in FLOWS:   
            print(bio_check_name)
    
            value = 1 if str(bio['categories'][0]) == "natural resource" else -1
            line = (bio.key, value)
            my_cfs_water.append(line)
    
    my_method = Method(new_water_name)
    my_metadata = {"unit": "cubic meter", "meaning": "to represent net water consumption"}
    my_method.register(**my_metadata)
    my_method.write(my_cfs_water)

('Water, well, in ground', ('natural resource', 'in water'))
('Water, lake', ('natural resource', 'in water'))
('Water', ('water', 'ground-'))
('Water, cooling, unspecified natural origin', ('natural resource', 'in water'))
('Water', ('water',))
('Water, unspecified natural origin', ('natural resource', 'in water'))
('Water, river', ('natural resource', 'in water'))
('Water', ('water', 'ground-, long-term'))
('Water, unspecified natural origin', ('natural resource', 'in ground'))
('Water', ('water', 'fossil well'))
('Water', ('water', 'surface water'))
('Water, turbine use, unspecified natural origin', ('natural resource', 'in water'))


### 1. BF-BOF using pig iron only
Get the database we would like to use to make a a new steel activity only producing primary steel with only pig iron

In [15]:
act_infos = [('steel production, converter, low-alloyed',	
            'steel, low-alloyed',''),
            ('steel production, converter, unalloyed',	
            'steel, unalloyed',''),
            ]

if GENERATE_ACTS:
    for act_info in act_infos:
        for i, db in enumerate(all_ei_databases):
            potential_locs = [act['location'] for act in bw.Database(db) if act['name'] == act_info[0]
                             and act['reference product'] == act_info[1] ]
            
            for loc in potential_locs:
                print(loc)
                if ('reference' in str(db)) and (loc == 'EUR'):
                    # those REMIND locations are not in the ref db
                    print(f"No activity found for name '{act_info[0]}' in location '{loc}' in database: {db}'")
                    continue
                else:
                    #loc_spec = "EUR" if 'reference' not in str(db) else loc
                    loc_spec= loc
                    # find bw act we want to copy and modify
                    act_init = find_bw_act(db, act_info[0], act_info[1], loc_spec)
                
                    new_name = f"{act_info[0]}, only primary steel" 
                    check_act = [act for act in bw.Database(db) if new_name == act['name'] and loc_spec == act['location']]
                
                    #copy and make new one
                    if len(check_act) == 0:
                        act = act_init.copy()
                    else:
                        print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,loc_spec))
                        #continue
                        act_delete = find_bw_act(db, new_name, act_info[1], loc_spec)
                        act_delete.delete()
                        act = act_init.copy()
                        
    
                    amount_iron = 0
                    # get total amounts of primary steel:
                    for exc in act.exchanges():
                        if exc['name']== 'pig iron production' or exc['name']== 'market for pig iron':
                            amount_iron += exc['amount']
                        elif exc['name']== 'market for iron scrap, sorted, pressed' or exc['name']== "market for iron scrap, unsorted":
                            amount_iron += exc['amount']
                        elif (exc['name']== 'market for sponge iron') or (exc['name']== 'sponge iron production'):
                            amount_iron += exc['amount']
                        else:
                            pass
    
                    print(f"Amount primary iron is {amount_iron}")
                    if amount_iron<0.01:
                        print(f"WARNING: no iron found for: {loc}, deleting act")
                        act.delete()
                        continue
                
                    act['name'] = new_name
                    act['location'] = loc_spec
                    act.save()
            
                    # Get the amount of CO2 directly emitted, as proxy for the factor used that is captured
                    for exc in act.exchanges():
                        if ('market for pig iron' == exc['name'] or exc['name']== 'pig iron production') and (exc['type']=='technosphere'):
                            print(f"Changed primary iron to {amount_iron} for {exc['name']}")
                            exc['amount'] = amount_iron
                            exc.save()
                        if ('market for iron scrap, sorted, pressed' == exc['name'] or exc['name']== "market for iron scrap, unsorted") and (exc['type']=='technosphere'):
                            print(f"Changed scrap iron amount to 0 for {exc['name']}")
                            exc['amount'] = 0
                            exc.save()
                        if (exc['name']== 'market for sponge iron') or (exc['name']== 'sponge iron production'):
                            print(f"Changed sponge iron to 0 for {exc['name']}")
                            exc['amount'] = 0
                            exc.save()
    
                    act.save()

IN
Amount primary iron is 1.18848
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Changed primary iron to 1.18848 for pig iron production
Changed sponge iron to 0 for sponge iron production
RoW
Amount primary iron is 1.0675898889860478
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Changed primary iron to 1.0675898889860478 for market for pig iron
Changed sponge iron to 0 for market for sponge iron
RER
Amount primary iron is 1.066809403
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Changed primary iron to 1.066809403 for market for pig iron
LAM
Amount primary iron is 1.0675898889860478
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Changed primary iron to 1.0675898889860478 for market for pig iron
Changed sponge iron to 0 for market for sponge iron
MEA
Amount primary iron is 1.0675898889860478
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Changed prima

### 2. DRI-EAF using sponge iron only

In [16]:
act_infos = [('steel production, electric, low-alloyed',	
            'steel, low-alloyed','')
            ]

if GENERATE_ACTS:
    for act_info in act_infos:
        for i, db in enumerate(all_ei_databases):
            potential_locs = [act['location'] for act in bw.Database(db) if act['name'] == act_info[0]
                             and act['reference product'] == act_info[1] ]
            
            for loc in potential_locs:
                if ('reference' in str(db)) and (loc == 'EUR'):
                    # those REMIND locations are not in the ref db
                    print(f"No activity found for name '{act_info[0]}' in location '{loc}' in database: {db}'")
                    continue
                else:
                    loc_spec = "EUR" if 'reference' not in str(db) else loc
                    loc_spec= loc
                    # find bw act we want to copy and modify
                    act_init = find_bw_act(db, act_info[0], act_info[1], loc_spec)
    
                    new_name = f"{act_info[0]}, primary steel, using direct reduced iron" 
                    check_act = [act for act in bw.Database(db) if new_name == act['name'] and loc_spec == act['location']]
                
                    #copy and make new one
                    if len(check_act) == 0:
                        act = act_init.copy()
                    else:
                        print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,loc_spec))
                        act_delete = find_bw_act(db, new_name, act_info[1], loc_spec)
                        act_delete.delete()
                        act = act_init.copy()
    
                    amount_iron = 0
                    # get total amounts of iron:
                    for exc in act.exchanges():
                        if exc['name']== 'pig iron production' or exc['name']== 'market for pig iron':
                            amount_iron += exc['amount']
                        elif exc['name']== 'market for iron scrap, sorted, pressed' or exc['name']== "market for iron scrap, unsorted":
                            amount_iron += exc['amount']
                        elif (exc['name']== 'market for sponge iron') or (exc['name']== 'sponge iron production'):
                            amount_iron += exc['amount']
                        else:
                            pass
    
                    print(f"Amount iron is {amount_iron}")
    
                    if amount_iron<0.01:
                        # this can happen in prospective LCA dbs, we dont need those acts
                        print(f"WARNING: no iron found for: {loc}, deleting act")
                        act.delete()
                        continue
                
                    act['name'] = new_name
                    act['location'] = loc_spec
                    act.save()
    
                    sponge_iron=0
                    # Get the amount of CO2 directly emitted, as proxy for the factor used that is captured
                    for exc in act.exchanges():
                        if ('market for pig iron' == exc['name'] or exc['name']== 'pig iron production') and (exc['type']=='technosphere'):
                            print(f"Changed primary iron to 0 for {exc['name']}")
                            exc['amount'] = 0
                            exc.save()
                        if ('market for iron scrap, sorted, pressed' == exc['name'] or exc['name']== "market for iron scrap, unsorted") and (exc['type']=='technosphere'):
                            print(f"Changed scrap iron amount to 0 for {exc['name']}")
                            exc['amount'] = 0
                            exc.save()
                        if (exc['name']== 'market for sponge iron') or (exc['name']== 'sponge iron production'):
                            print(f"Changed sponge iron to {amount_iron} for {exc['name']}")
                            exc['amount'] = amount_iron
                            exc.save()
                            sponge_iron=1
    
                    # add separately an activity representing direct reduced iron inclusion
                    if sponge_iron ==0:
                        # This is the only activity with sponge iron, aware that this is for India though (the market only consists of India for DRI).
                        print(f"Manually add direct reduced (sponge) iron to EAF for {act['name']} [{act['location']}]")
                        add_act = find_bw_act(db, 'market for sponge iron','sponge iron', 'GLO')
                
                        # Add the exchange
                        act.new_exchange(input=add_act.key, amount=amount_iron, 
                                             name = add_act['name'],
                                             product = add_act['reference product'],
                                        location = add_act['location'], unit=add_act['unit'],type='technosphere').save()
    
                    act.save()

Amount iron is 1.1363636363636365
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Manually add direct reduced (sponge) iron to EAF for steel production, electric, low-alloyed, primary steel, using direct reduced iron [CH]
Amount iron is 1.259267158
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Changed primary iron to 0 for pig iron production
Changed sponge iron to 1.259267158 for sponge iron production
Amount iron is 1.13139206290304
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Manually add direct reduced (sponge) iron to EAF for steel production, electric, low-alloyed, primary steel, using direct reduced iron [Europe without Switzerland and Austria]
Amount iron is 1.1660082593311363
Changed scrap iron amount to 0 for market for iron scrap, sorted, pressed
Changed scrap iron amount to 0 for market for iron scrap, unsorted
Changed primary iron to 0 for market for pig iron
Changed sponge iron to 1.1660082

caz_acts = [act for act in bw.Database('ecoinvent_remind_SSP2-PkBudg1150_2030_base') if act.key == ('ecoinvent_remind_SSP2-PkBudg1150_2030_base', '1d6709a7a491408dbc8cf8c2fa832553')]
caz_acts[0].delete()

### 3. EAF using scrap iron only (secondary steel)

In [17]:
act_infos = [('steel production, electric, low-alloyed',	
            'steel, low-alloyed','')
            ]

if GENERATE_ACTS:
    for act_info in act_infos:
        for i, db in enumerate(all_ei_databases):
            potential_locs = [act['location'] for act in bw.Database(db) if act['name'] == act_info[0]
                             and act['reference product'] == act_info[1] ]
            
            for loc in potential_locs:
                if ('reference' in str(db)) and (loc == 'EUR'):
                    # those REMIND locations are not in the ref db
                    print(f"No activity found for name '{act_info[0]}' in location '{loc}' in database: {db}'")
                    continue
                else:
                    loc_spec = "EUR" if 'reference' not in str(db) else loc
                    loc_spec= loc
                    # find bw act we want to copy and modify
                    act_init = find_bw_act(db, act_info[0], act_info[1], loc_spec)
    
                    new_name = f"{act_info[0]}, secondary steel, using scrap iron" 
                    check_act = [act for act in bw.Database(db) if new_name == act['name'] and loc_spec == act['location']]
                
                    #copy and make new one
                    if len(check_act) == 0:
                        act = act_init.copy()
                    else:
                        print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,loc_spec))
                        act_delete = find_bw_act(db, new_name, act_info[1], loc_spec)
                        act_delete.delete()
                        act = act_init.copy()
    
                    amount_iron = 0
                    # get total amounts of iron:
                    for exc in act.exchanges():
                        if exc['name']== 'pig iron production' or exc['name']== 'market for pig iron':
                            amount_iron += exc['amount']
                        elif exc['name']== 'market for iron scrap, sorted, pressed' or exc['name']== "market for iron scrap, unsorted":
                            amount_iron += exc['amount']
                        elif (exc['name']== 'market for sponge iron') or (exc['name']== 'sponge iron production'):
                            amount_iron += exc['amount']
                        else:
                            pass
    
                    print(f"Amount iron is {amount_iron}")
    
                    if amount_iron<0.01:
                        # this can happen in prospective LCA dbs, we dont need those acts
                        print(f"WARNING: no iron found for: {loc}, deleting act")
                        act.delete()
                        continue
                
                    act['name'] = new_name
                    act['location'] = loc_spec
                    act.save()
    
                    scrap_iron=0
                    # Get the amount of CO2 directly emitted, as proxy for the factor used that is captured
                    for exc in act.exchanges():
                        if ('market for pig iron' == exc['name'] or exc['name']== 'pig iron production') and (exc['type']=='technosphere'):
                            print(f"Changed primary iron to 0 for {exc['name']}")
                            exc['amount'] = 0
                            exc.save()

                        if 'iron scrap' in exc['name']:
                            if ('market for iron scrap, sorted, pressed' == exc['name']):
                                exc['amount'] = amount_iron if scrap_iron == 0 else 0
                                
                                if scrap_iron == 0:
                                    print(f"Changed scrap iron amount to {amount_iron} for {exc['name']}")
                                    
                                exc.save()
                                scrap_iron=1
                            elif(exc['name']== "market for iron scrap, unsorted") and (exc['type']=='technosphere'):
                                exc['amount'] = amount_iron if scrap_iron == 0 else 0
                                
                                if scrap_iron == 0:
                                    print(f"Changed scrap iron amount to {amount_iron} for {exc['name']}")
                                    
                                exc.save()
                                scrap_iron=1 
                        if (exc['name']== 'market for sponge iron') or (exc['name']== 'sponge iron production'):
                            exc['amount'] = 0
                            exc.save()
    
                    # add separately an activity representing direct reduced iron inclusion
                    if scrap_iron ==0:
                        # This is the only activity with sponge iron, aware that this is for India though (the market only consists of India for DRI).
                        print(f"Manually add iron scrap, unsorted to EAF for {act['name']} [{act['location']}]")
                        add_act = find_bw_act(db, 'market for iron scrap, unsorted', 'iron scrap, unsorted', 'GLO')
                
                        # Add the exchange
                        act.new_exchange(input=add_act.key, amount=amount_iron, 
                                             name = add_act['name'],
                                             product = add_act['reference product'],
                                         location = add_act['location'], unit=add_act['unit'],type='technosphere').save()
    
                    act.save()

Amount iron is 1.13139206290304
Changed scrap iron amount to 1.13139206290304 for market for iron scrap, sorted, pressed
Amount iron is 1.259267158
Changed scrap iron amount to 1.259267158 for market for iron scrap, sorted, pressed
Changed primary iron to 0 for pig iron production
Amount iron is 1.1660082593311363
Changed scrap iron amount to 1.1660082593311363 for market for iron scrap, sorted, pressed
Changed primary iron to 0 for market for pig iron
Amount iron is 1.1363636363636365
Changed scrap iron amount to 1.1363636363636365 for market for iron scrap, sorted, pressed
Amount iron is 1.120889101
Changed scrap iron amount to 1.120889101 for market for iron scrap, sorted, pressed
Amount iron is 1.1026658100979136
Changed scrap iron amount to 1.1026658100979136 for market for iron scrap, unsorted
Amount iron is 0
Amount iron is 0
Amount iron is 0
Amount iron is 1.259267158
Changed scrap iron amount to 1.259267158 for market for iron scrap, sorted, pressed
Changed primary iron to 0 f

## Make initital activities for Figure 4 in the manuscript including long-distance transport

# Static transport mode metadata (shared across all continents)
transport_modes = {
    "train": {
        "name": "market group for transport, freight train",
        "reference product": "transport, freight train",
        "unit": "ton kilometer",
        'km': 500
    },
    "ship": {
        "name": "market for transport, freight, sea, container ship",
        "reference product": "transport, freight, sea, container ship",
        "unit": "ton kilometer",
        'km': 10000
    }
}

if GENERATE_ACTS:
    for i, db in enumerate(all_ei_databases):
        steel_activities = [ act for act in bd.Database(db) 
                             if ('steel production, unalloyed' in act['name'] and
                             'GLO' == act['location']) or ('steel production, converter, unalloyed, only primary steel' in act['name'] and
                             act['location'] in ['RoW', 'World']) or ('primary steel, using direct reduced iron' in act['name'] and
                             act['location'] in ['RoW', 'World']) or ('steel production, electric, low-alloyed, secondary steel, using scrap iron' in act['name'] and
                             act['location'] in ['RoW', 'World'])] 
    
        for j, act_sel in enumerate(steel_activities):
            print(db,j)
            new_name = f"{act_sel['name']}, with transport" 
            check_act = [act for act in bw.Database(db) if new_name == act['name'] and act_sel['location'] == act['location']
                        and  act_sel['reference product'] == act['reference product']
                        ]
        
            #copy and make new one
            if len(check_act) == 0:
                act = act_sel.copy()
            else:
                print("WARNING: Activity '{}'[{}] already generated, deleting!".format(new_name,act_sel['location']))
                act_delete = find_bw_act(db, new_name, act_sel['reference product'], act_sel['location'])
                act_delete.delete()
                act = act_sel.copy()

            act['name'] = new_name

            # Add the two transport exchanges
            for mode, exchange_info in transport_modes.items():
                add_act = find_bw_act(db, exchange_info['name'], exchange_info['reference product'], 'GLO')
                
                act.new_exchange(input=add_act.key, amount=exchange_info['km']*1e-3, 
                                     name = add_act['name'],
                                     product = add_act['reference product'],
                                 location = add_act['location'], unit=add_act['unit'],type='technosphere').save()
        
                act.save()